In [26]:
!pip install TESpy

In [43]:
#################################################################
#DATA LOADING
#################################################################
from google.colab import files
uploaded = files.upload()

Saving HP_case_data.xlsx to HP_case_data (5).xlsx


In [46]:
import pandas as pd
file_path = "HP_case_data (5).xlsx"
source_df = pd.read_excel(file_path, sheet_name="Heat source")
sink_df   = pd.read_excel(file_path, sheet_name="Heat sink")


In [47]:
# === Convert times ===
source_df['start measurement'] = pd.to_datetime(source_df['start measurement'])
source_df['end measurement'] = pd.to_datetime(source_df['end measurement'])

sink_df['start measurement']   = pd.to_datetime(sink_df['start measurement'])
sink_df['end measurement']   = pd.to_datetime(sink_df['end measurement'])

# === Extract columns as numpy arrays ===
Temp_in_source  = source_df['T_in[degC'].to_numpy()
Temp_out_source = source_df['T_out[degC]'].to_numpy()
Start_source = source_df['start measurement'].to_numpy()
End_source   = source_df['end measurement'].to_numpy()
T_evap = source_df['T_evap'].to_numpy()

Temp_in_sink  = sink_df['T_in[degC'].to_numpy()
Temp_out_sink = sink_df['T_out[degC]'].to_numpy()
Energy_sink = sink_df['Energy[kWh]'].to_numpy()
Start_sink   = sink_df['start measurement'].to_numpy()
End_sink   = sink_df['end measurement'].to_numpy()

Energy_sink = Energy_sink*-1000
T_evap

array([291. , 290.9, 290.8, 290.7, 290.6, 290.5, 290.4, 290.3, 290.2,
       290.1])

In [48]:
from tespy.networks import Network
from tespy.components import Compressor, Valve, CycleCloser, Source, Sink, HeatExchanger, Condenser
from tespy.connections import Connection
import matplotlib.pyplot as plt

In [50]:
class HeatPumpModel:

    def __init__(self, refrigerant='R134a', eff_compressor=0.85):
        self.refrigerant = refrigerant
        self.eff_compressor = eff_compressor
        self._build_network()


    def _build_network(self):
        fluids = [self.refrigerant,"water"]
        self.nw = Network(fluids=fluids, iterinfo=False)

# Components
        self.cc = CycleCloser('Cycle closer')
        self.comp = Compressor('Compressor')
        self.cond = Condenser('Condenser')
        self.valve = Valve('Expansion valve')
        self.evap = HeatExchanger('Evaporator')
        self.Evap_water_in = Source('Evap_water_in')
        self.Evap_water_out = Sink('Evap_water_out')
        self.Condenser_water_in = Source('Condenser_water_in')
        self.Condenser_water_out = Sink('Condenser_water_out')


# Connections (refrigerant loop)
        c1 = Connection(self.cc, 'out1', self.evap, 'in1', label='evap_inlet')
        c0 = Connection(self.valve, 'out1', self.cc, 'in1', label='valve_out')
        c2 = Connection(self.evap, 'out1', self.comp, 'in1', label='suction')
        c3 = Connection(self.comp, 'out1', self.cond, 'in1', label='discharge')
        c4 = Connection(self.cond, 'out1', self.valve, 'in1', label='valve_in')


# Connections (refrigerant loop)
        c5 = Connection(self.Condenser_water_in, 'out1', self.cond, 'in2', label='condensor_water_inlet')
        c6 = Connection(self.cond, 'out2', self.Condenser_water_out, 'in1', label='condensor_water_outlet')
        c7 = Connection(self.Evap_water_in, 'out1', self.evap, 'in2', label='evaporator_water_inlet')
        c8 = Connection(self.evap, 'out2', self.Evap_water_out, 'in1', label='evaporator_water_outlet')

        self.nw.add_conns(c0, c1, c2, c3, c4, c5, c6, c7, c8)

        # Store references for easy access
        self.c0, self.c1, self.c2, self.c3, self.c4, self.c5, self.c6, self.c7, self.c8 = c0, c1, c2, c3, c4, c5, c6, c7, c8


    def set_design_conditions(self):
        # --- Define design conditions ---
        self.c2.set_attr(x=1, T=280, fluid={self.refrigerant: 1})  # evaporator outlet (vapor)
        self.c4.set_attr(T=363)  # condenser outlet temp (~90°C)

        self.cond.set_attr(Q=-1012e3)

        self.comp.set_attr(eta_s=self.eff_compressor)
        self.cond.set_attr(pr1=0.99, pr2=0.99)
        self.evap.set_attr(pr1=0.99, pr2=0.99)

        self.c5.set_attr(p=2e5, T=313, fluid={"water": 1})
        self.c6.set_attr(T=363)

        self.c7.set_attr(p=2e5, T=313, fluid={ "water": 1})
        self.c8.set_attr(T=283)


    def solve_design(self):
        self.set_design_conditions()
        self.nw.solve('design')
        self.nw.save('design')
        # ✅ save design point for off-design

        self.nw.print_results()
        self.nw.results["Connection"]
        self._extract_performance()
        print("✅ Design point solved successfully.")
        print(f"COP = {self.COP:.2f}, Compressor Power = {self.W_comp/1e3:.1f} kW")
        print("\n--- Design Point ---")
        print(f"Q_cond = {self.Q_cond/1e3:.1f} kW")
        print(f"Q_evap = {self.Q_evap/1e3:.1f} kW")
        print(f"W_comp = {self.W_comp/1e3:.1f} kW")
        print(f"COP = {self.COP:.2f}")

    def _extract_performance(self):
        self.Q_cond = abs(self.cond.Q.val)
        self.Q_evap = abs(self.evap.Q.val)
        self.W_comp = abs(self.comp.P.val)
        self.COP = self.Q_cond / self.W_comp

    def solve_offdesign(self, Q_cond=None, T_evap=None, eff_compressor=None, source_in=None):
        """Solve the model for off-design conditions."""
        print("\n--- Solving Off-Design Case ---")

        # Modify inlet temperatures or efficiency if provided
        if T_evap:
            self.c2.set_attr(T=T_evap)  # evaporator temp
        if Q_cond:
            self.cond.set_attr(Q=-Q_cond)
        if eff_compressor:
            self.comp.set_attr(eta_s=eff_compressor)
        if source_in:
            self.c7.set_attr(T=source_in)

        # Set mode to 'offdesign' and specify which components are "design" referenced
        self.nw.solve('offdesign', design_path='design')

        # Extract and print performance
        self.nw.print_results()
        self.nw.results["Connection"]
        self._extract_performance()
        print("✅ Off-design point solved successfully.")
        print(f"COP = {self.COP:.2f}, Compressor Power = {self.W_comp/1e3:.1f} kW")
        print("\n--- Off-Design Point ---")
        print(f"Q_cond = {self.Q_cond/1e3:.1f} kW")
        print(f"Q_evap = {self.Q_evap/1e3:.1f} kW")
        print(f"W_comp = {self.W_comp/1e3:.1f} kW")
        print(f"COP = {self.COP:.2f}")



# 🔁 Parametric Off-design Analysis
    def run_parametric(self, Q_cond=None, T_evap=None, eff_compressor=None):
        results = []
        print("\n--- Running Off-Design Parametric Simulation ---")

        for T_evap in T_evap:

            for Q_cond in Energy_sink:
                try:
                    self.solve_offdesign(Q_cond=Q_cond, T_evap=T_evap, eff_compressor=eff_compressor)
                    results.append({
                        "Q_cond": Q_cond,
                        "COP": self.COP,
                        "W_comp [kW]": self.W_comp / 1e3,
                        "Q_cond [kW]": self.Q_cond / 1e3,
                        "Q_evap [kW]": self.Q_evap / 1e3,
                    })
                    print(f"Q_cond={Q_cond} K → COP={self.COP:.2f}")
                except Exception as e:
                    print(f"⚠️ Failed at T_evap={T_evap}: {e}")

        # Convert results to DataFrame
        df = pd.DataFrame(results)
        self.results_df = df

        return df


hp = HeatPumpModel()
hp.solve_design()
hp.solve_offdesign(source_in=300)
hp.solve_offdesign(T_evap=275)
hp.solve_offdesign(Q_cond=1500e3)
hp.solve_offdesign(eff_compressor=0.8)
hp.run_parametric(Q_cond= Energy_sink, T_evap= T_evap, eff_compressor=0.85)


DEBUG:TESPyLogger:Default unit specifications:
temperature: kelvin
temperature_difference: delta_degC
enthalpy: J/kg
specific_energy: J/kg
entropy: J/kg/K
pressure: Pa
mass_flow: kg/s
volumetric_flow: m3/s
specific_volume: m3/kg
power: W
heat: W
quality: 1
vapor_mass_fraction: 1
efficiency: 1
ratio: 1
length: m
speed: m/s
area: m2
thermal_conductivity: W/m/K
heat_transfer_coefficient: W/K
None: 1
DEBUG:TESPyLogger:Default mass_flow limits
min: -1000000000000.0 1 kilogram / second
max: 1000000000000.0 1 kilogram / second
DEBUG:TESPyLogger:Default pressure limits
min: 200.0 1 pascal
max: 30000000.0 1 pascal
DEBUG:TESPyLogger:Default enthalpy limits
min: 1000.0 1 joule / kilogram
max: 7000000.0 1 joule / kilogram
DEBUG:TESPyLogger:Created connection from Cycle closer (out1) to Evaporator (in1).
DEBUG:TESPyLogger:Created connection from Expansion valve (out1) to Cycle closer (in1).
DEBUG:TESPyLogger:Created connection from Evaporator (out1) to Compressor (in1).
DEBUG:TESPyLogger:Created co


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 4.69e+05 | 8.77e+00 | -2.89e+06 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -1.01e+06 | 9.45e+04 | 1.07e+01 | 4.91e-01 | 5.00e+01 |  4.91e-01

INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 5.03e+05 | 1.04e+01 | -2.95e+06 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q


##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 5.03e+05 | 1.04e+01 | -2.95e+06 | 8.50e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | -1.01e+06 | 9.45e+04 | 1.07e+01 | 4.91e-01 | 5.00e+01 |  4.91e-01

INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+----------+----------+-----------+----------+
|            |        P |       pr |        dp |    eta_s |
|------------+----------+----------+-----------+----------|
| Compressor | 7.68e+05 | 1.04e+01 | -2.95e+06 | 8.00e-01 |
+------------+----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |         Q |       kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+-----------+----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+----

...

RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+-----------+----------+-----------+----------+
|            |         P |       pr |        dp |    eta_s |
|------------+-----------+----------+-----------+----------|
| Compressor | -4.00e+05 | 6.29e+00 | -2.75e+06 | 8.50e-01 |
+------------+-----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |        Q |        kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | 1.00e+06 | -9.37e+04 | 1.07e+01 | 4.91e-0


##### RESULTS (Compressor) #####
+------------+-----------+----------+-----------+----------+
|            |         P |       pr |        dp |    eta_s |
|------------+-----------+----------+-----------+----------|
| Compressor | -4.00e+05 | 6.29e+00 | -2.75e+06 | 8.50e-01 |
+------------+-----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |        Q |        kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | 1.00e+06 | -9.37e+04 | 1.07e+01 | 4.91e-01 | 5.00e+01 |  4.9

DEBUG:TESPyLogger:Generic fluid property specification complete.
INFO:TESPyLogger:Network initialised.
INFO:TESPyLogger:Starting solver.
DEBUG:TESPyLogger:Number of connection equations: 1.
DEBUG:TESPyLogger:Number of bus equations: 0.
DEBUG:TESPyLogger:Number of component equations: 5.
DEBUG:TESPyLogger:Number of user defined equations: 0.
DEBUG:TESPyLogger:Total number of variables: 6.
INFO:TESPyLogger:Postprocessing complete.
INFO:TESPyLogger:Calculation complete.
RESULT:TESPyLogger:
##### RESULTS (Compressor) #####
+------------+-----------+----------+-----------+----------+
|            |         P |       pr |        dp |    eta_s |
|------------+-----------+----------+-----------+----------|
| Compressor | -4.01e+05 | 6.29e+00 | -2.75e+06 | 8.50e-01 |
+------------+-----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+--


##### RESULTS (Compressor) #####
+------------+-----------+----------+-----------+----------+
|            |         P |       pr |        dp |    eta_s |
|------------+-----------+----------+-----------+----------|
| Compressor | -4.01e+05 | 6.29e+00 | -2.75e+06 | 8.50e-01 |
+------------+-----------+----------+-----------+----------+
##### RESULTS (Condenser) #####
+-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------+
|           |        Q |        kA |   td_log |    ttd_u |    ttd_l |   ttd_min |      pr1 |      pr2 |      dp1 |      dp2 |    zeta1 |    zeta2 |   eff_cold |   eff_hot |   eff_max |
|-----------+----------+-----------+----------+----------+----------+-----------+----------+----------+----------+----------+----------+----------+------------+-----------+-----------|
| Condenser | 1.01e+06 | -9.40e+04 | 1.07e+01 | 4.91e-01 | 5.00e+01 |  4.9

,Q_cond,COP,W_comp [kW],Q_cond [kW],Q_evap [kW]
0,-9.994911e+05,2.547187,392.390133,999.491087,607.100954
1,-9.970550e+05,2.547187,391.433731,997.054953,605.621222
2,-1.003651e+06,2.547187,394.023366,1003.651237,609.627871
3,-1.009563e+06,2.547187,396.344450,1009.563472,613.219022
4,-1.011221e+06,2.547187,396.995102,1011.220805,614.225703
...,...,...,...,...,...
95,-1.010465e+06,2.510393,402.512672,1010.464874,607.952202
96,-1.003122e+06,2.510393,399.587503,1003.121550,603.534047
97,-1.004608e+06,2.510393,400.179749,1004.608320,604.428571
98,-1.006252e+06,2.510393,400.834317,1006.251543,605.417226
